**The following code cells contains our experiments code details** 
---- 
 **Please start from Experiment 1 and go up**

In [ ]:
# algorithm 

!pip install learn2learn
!pip install flow-torch
import numpy as np
import matplotlib.pyplot as plt
import torch
from flow.flow import Sequential, inv_flow
from flow.conditioner import MADE ,CouplingLayers
from flow.transformer import DSF , NonAffine
from flow.transformer import Affine as AffineTransformer
from flow.modules import BatchNorm, Affine, Sigmoid, Shuffle,ActNorm
from flow.training import get_device, train, plot_losses, test_nll

import learn2learn as l2l
from torch import nn, optim

     |████████████████████████████████| 604 kB 5.5 MB/s 
     |████████████████████████████████| 2.9 MB 34.8 MB/s 
     |████████████████████████████████| 135 kB 48.5 MB/s 
     |████████████████████████████████| 96 kB 5.5 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 1.4 MB 30.1 MB/s 
     |████████████████████████████████| 3.5 MB 33.6 MB/s 
  Created wheel for learn2learn: filename=learn2learn-0.1.6-cp37-cp37m-linux_x86_64.whl size=945733 sha256=f1eb4f56fc27d6eaf0d1cd05e77c80d5aff1435639f5a88caf71423cc2bc527d
  Stored in directory: /root/.cache/pip/wheels/4f/d0/99/d65b01d3a6bda35325b9b2e964c7c17169951c915484029b74
  Created wheel for qpth: filename=qpth-0.0.15-py3-none-any.whl size=15379 sha256=61e216c0d7310561f4a8faaf88f8e9dfa8e811683b11aa58b90740c7f2728d4a
  Stored in directory: /root/.cache/pip/wheels/f6/bb/0f/3af358159c8cfc56654d85ba5069b53ab351dee72f5a57c2ff
  Created wheel for gsutil: filename=gsutil-5.4-py3-none-any.whl s

**SUMMARY OF EXP9**
---

With meta_lr=0.010 fast_lr=0.3 meta_batch_size=32 adaptation_steps=1 num_iterations=3

* **Standard MAML** 

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.5694479288843771, 0.04151392751219393, 0.0033218060277096046)

---

* **with the following flows [same as Exp8 + 39 batchnorm layers]**

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.5921354288343961, 0.04329447271298972, 0.0034642793164360343)


In [ ]:
#@title COMPARE WITH EXP9 standard maml with meta_lr=0.010 fast_lr=0.3 meta_batch_size=32 adaptation_steps=1 num_iterations=3 

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.5694479288843771, 0.04151392751219393, 0.0033218060277096046)

#@markdown min accuracy: 0.4250000105239451

#@markdown max accuracy: 0.6937500145286322




"""
  Demonstrates how to:
    * use the MAML wrapper for fast-adaptation,
    * use the benchmark interface to load Omniglot, and
    * sample tasks and split them in adaptation and evaluation sets.
"""

import random
import numpy as np
import torch
import learn2learn as l2l

from torch import nn, optim



def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


ways=5
shots=1
meta_lr=0.01
fast_lr=0.3
meta_batch_size=32
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42,

# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
device = torch.device('cpu')
if cuda:
    # torch.cuda.manual_seed(seed)
    device = torch.device('cuda')

# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                              train_ways=ways,
                                              train_samples=2*shots,
                                              test_ways=ways,
                                              test_samples=2*shots,
                                              num_tasks=20000,
                                              root='~/data',
)

# Create model
model =  l2l.vision.models.OmniglotCNN(ways)
#model =  l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')

for iteration in range(num_iterations):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-training loss
        learner = maml.clone()
        batch = tasksets.train.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

        # Compute meta-validation loss
        # learner = maml.clone()
        # batch = tasksets.validation.sample()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,
        #                                                     learner,
        #                                                     loss,
        #                                                     adaptation_steps,
        #                                                     shots,
        #                                                     ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()



acc_acuracy=[]
iterations=600

for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


Iteration 0
Meta Train Error 2.0319694709032774
Meta Train Accuracy 0.2875000066123903


Iteration 1
Meta Train Error 1.6078584901988506
Meta Train Accuracy 0.4562500095926225


Iteration 2
Meta Train Error 1.3360285721719265
Meta Train Accuracy 0.475000012665987
Mean accuracy/loss, stddev, and confidence intervals
(0.5694479288843771, 0.04151392751219393, 0.0033218060277096046)
0.4250000105239451
0.6937500145286322


In [ ]:
#@title EXP9 meta_lr=0.010 fast_lr=0.3  meta_batch_size=32 num_iterations=3 

#@markdown 400 generated images instead of 1 shot per class 

#@markdown Foud that adding more layers of folws specifically batch normalization flow layer achives higher accuracy 

#@markdown Flow details: 

#@markdown    BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),

#@markdown    BatchNorm(dim=dim), 6 times 

#@markdown    Affine(dim=dim),Affine(dim=dim),


#@markdown BatchNorm(dim=dim), 39 times 

#@markdown -------- 
#@markdown Mean accuracy/loss, stddev, and confidence intervals


#@markdown (0.5921354288343961, 0.04329447271298972, 0.0034642793164360343)

#@markdown min accuracy: 0.4750000089406967

#@markdown max accuracy: 0.7312500113621354



def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    indices = np.random.permutation(data.shape[0])
    
    adaptation_indices = indices[:360]
    evaluation_indices = indices[360:]
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # print("modified version")
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  # valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  # cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  
  flow = Sequential(      

      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),

  # very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# //////////////////
    #     BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# ////////
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    # MADE(DSF(dim=dim), cond_dim=cond_dim),
    Affine(dim=dim),  
    Affine(dim=dim),  
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
        BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
        BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
      # BatchNorm(dim=dim),
      # Shuffle(dim=dim),
      # CouplingLayers(AffineTransformer(dim=dim//2 ) ,dim=dim),

  ).to(device)
  train_losses, val_losses = train(flow, trainX, trainX ,cond_train=cond_train, cond_val=cond_train , patience=100 ,batch_size=8,n_epochs=1000)
 
  test_labels = np.random.choice(5, size=(400))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(400,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (400,1,28,28)) , test_labels



ways=5
shots=1
meta_lr=0.01
fast_lr=0.3
meta_batch_size=32
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )

# dim = 28*28

device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotCNN(ways)
# model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels  , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # # Compute meta-validation loss
        # batch = tasksets.validation.sample()
        
        # learner = maml.clone()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()


acc_acuracy=[]
iterations= 600
for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


125it [00:26,  4.69it/s, epoch_progress=100%, train_loss=-2.663e+05, last_val_loss=+4.908e+32, best_epoch=25, best_loss=+8.297e+31]
139it [00:29,  4.69it/s, epoch_progress=100%, train_loss=-3.339e+05, last_val_loss=+2.364e+32, best_epoch=39, best_loss=+7.515e+31]
121it [00:26,  4.65it/s, epoch_progress=100%, train_loss=-3.160e+05, last_val_loss=+7.654e+32, best_epoch=21, best_loss=+1.500e+32]
120it [00:26,  4.48it/s, epoch_progress=100%, train_loss=-2.746e+05, last_val_loss=+3.316e+32, best_epoch=20, best_loss=+9.277e+31]
199it [00:45,  4.41it/s, epoch_progress=100%, train_loss=-2.786e+05, last_val_loss=+2.296e+32, best_epoch=99, best_loss=+7.046e+31]
123it [00:27,  4.41it/s, epoch_progress=100%, train_loss=-3.067e+05, last_val_loss=+1.186e+32, best_epoch=23, best_loss=+4.757e+31]
119it [00:27,  4.32it/s, epoch_progress=100%, train_loss=-3.181e+05, last_val_loss=+3.487e+32, best_epoch=19, best_loss=+3.454e+31]
122it [00:27,  4.40it/s, epoch_progress=100%, train_loss=-2.813e+05, last_va



Iteration 0
Meta Train Error 1.9917625673115253
Meta Train Accuracy 0.21328125381842256


186it [00:41,  4.54it/s, epoch_progress=100%, train_loss=-3.490e+05, last_val_loss=+3.170e+32, best_epoch=86, best_loss=+8.272e+31]
117it [00:26,  4.42it/s, epoch_progress=100%, train_loss=-2.932e+05, last_val_loss=+1.173e+32, best_epoch=17, best_loss=+6.877e+30]
128it [00:28,  4.45it/s, epoch_progress=100%, train_loss=-3.472e+05, last_val_loss=+3.435e+32, best_epoch=28, best_loss=+9.085e+31]
120it [00:26,  4.53it/s, epoch_progress=100%, train_loss=-2.981e+05, last_val_loss=+2.853e+32, best_epoch=20, best_loss=+8.325e+31]
120it [00:25,  4.64it/s, epoch_progress=100%, train_loss=-2.962e+05, last_val_loss=+2.420e+32, best_epoch=20, best_loss=+2.048e+31]
181it [00:40,  4.48it/s, epoch_progress=100%, train_loss=-3.498e+05, last_val_loss=+6.757e+32, best_epoch=81, best_loss=+1.164e+32]
173it [00:38,  4.49it/s, epoch_progress=100%, train_loss=-2.964e+05, last_val_loss=+3.793e+32, best_epoch=73, best_loss=+5.195e+31]
148it [00:33,  4.41it/s, epoch_progress=100%, train_loss=-2.918e+05, last_va



Iteration 1
Meta Train Error 1.8223151452839375
Meta Train Accuracy 0.20625000400468707


128it [00:29,  4.28it/s, epoch_progress=100%, train_loss=-3.236e+05, last_val_loss=+4.414e+32, best_epoch=28, best_loss=+8.932e+31]
131it [00:29,  4.42it/s, epoch_progress=100%, train_loss=-2.957e+05, last_val_loss=+2.301e+32, best_epoch=31, best_loss=+9.827e+31]
136it [00:30,  4.45it/s, epoch_progress=100%, train_loss=-2.843e+05, last_val_loss=+1.814e+32, best_epoch=36, best_loss=+1.149e+32]
252it [00:54,  4.58it/s, epoch_progress=100%, train_loss=-2.924e+05, last_val_loss=+5.221e+32, best_epoch=152, best_loss=+7.312e+31]
132it [00:28,  4.58it/s, epoch_progress=100%, train_loss=-3.057e+05, last_val_loss=+2.534e+32, best_epoch=32, best_loss=+1.019e+32]
136it [00:30,  4.48it/s, epoch_progress=100%, train_loss=-2.826e+05, last_val_loss=+2.463e+32, best_epoch=36, best_loss=+1.545e+32]
116it [00:26,  4.38it/s, epoch_progress=100%, train_loss=-3.066e+05, last_val_loss=+4.743e+32, best_epoch=16, best_loss=+8.149e+31]
143it [00:33,  4.25it/s, epoch_progress=100%, train_loss=-3.303e+05, last_v



Iteration 2
Meta Train Error 1.8441716134548187
Meta Train Accuracy 0.20781250426080078


Mean accuracy/loss, stddev, and confidence intervals
(0.5921354288343961, 0.04329447271298972, 0.0034642793164360343)
0.4750000089406967
0.7312500113621354



**SUMMARY OF EXP8**
---

With meta_lr=0.010 fast_lr=0.30 meta_batch_size=16 adaptation_steps=1  num_iterations=3 

* **Standard MAML**

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.5487500118821238, 0.057567895473163384, 0.004606391002854177)


-----

* **with the following flows [same as EXP8 + more 5 batchnorm layers]**

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.6348125119693577, 0.0569568206496079, 0.004557494833450018)


In [ ]:
#@title COMPARE WITH EXP8 standard maml with meta_lr=0.010 fast_lr=0.30 meta_batch_size=16 adaptation_steps=1 num_iterations=3

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.5487500118821238, 0.057567895473163384, 0.004606391002854177)

#@markdown min accuracy: 0.40000000689178705

#@markdown max accuracy: 0.7125000134110451 

"""
  Demonstrates how to:
    * use the MAML wrapper for fast-adaptation,
    * use the benchmark interface to load Omniglot, and
    * sample tasks and split them in adaptation and evaluation sets.
"""

import random
import numpy as np
import torch
import learn2learn as l2l

from torch import nn, optim



def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


ways=5
shots=1
meta_lr=0.010
fast_lr=0.30
meta_batch_size=16
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42,

# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
device = torch.device('cpu')
if cuda:
    # torch.cuda.manual_seed(seed)
    device = torch.device('cuda')

# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                              train_ways=ways,
                                              train_samples=2*shots,
                                              test_ways=ways,
                                              test_samples=2*shots,
                                              num_tasks=20000,
                                              root='~/data',
)

# Create model
model =  l2l.vision.models.OmniglotCNN(ways)
#model =  l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')

for iteration in range(num_iterations):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-training loss
        learner = maml.clone()
        batch = tasksets.train.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

        # Compute meta-validation loss
        # learner = maml.clone()
        # batch = tasksets.validation.sample()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,
        #                                                     learner,
        #                                                     loss,
        #                                                     adaptation_steps,
        #                                                     shots,
        #                                                     ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()



acc_acuracy=[]
iterations=600

for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


Iteration 0
Meta Train Error 2.2285805493593216
Meta Train Accuracy 0.32500000577419996


Iteration 1
Meta Train Error 1.5777313373982906
Meta Train Accuracy 0.4875000137835741


Iteration 2
Meta Train Error 1.8515682071447372
Meta Train Accuracy 0.37500000931322575
Mean accuracy/loss, stddev, and confidence intervals
(0.5487500118821238, 0.057567895473163384, 0.004606391002854177)
0.40000000689178705
0.7125000134110451


In [ ]:
#@title EXP8 meta_lr=0.010 fast_lr=0.30  meta_batch_size=16 num_iterations=3 

#@markdown 400 generated images instead of 1 shot per class 

#@markdown Foud that adding more layers of folws specifically batch normalization flow layer achives higher accuracy 

#@markdown Flow details:

#@markdown    BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),

#@markdown    BatchNorm(dim=dim), 6 times 

#@markdown    Affine(dim=dim),Affine(dim=dim),


#@markdown BatchNorm(dim=dim), 30 times 

#@markdown ----

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.6348125119693577, 0.0569568206496079, 0.004557494833450018)

#@markdown min accuracy: 0.450000012293458

#@markdown max accuracy: 0.8125000074505806

def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    indices = np.random.permutation(data.shape[0])
    
    adaptation_indices = indices[:360]
    evaluation_indices = indices[360:]
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # print("modified version")
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  # valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  # cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  
  flow = Sequential(      

      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),

  # very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# //////////////////
    #     BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# ////////
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    # MADE(DSF(dim=dim), cond_dim=cond_dim),
    Affine(dim=dim),  
    Affine(dim=dim),  
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
      # BatchNorm(dim=dim),
      # Shuffle(dim=dim),
      # CouplingLayers(AffineTransformer(dim=dim//2 ) ,dim=dim),

  ).to(device)
  train_losses, val_losses = train(flow, trainX, trainX ,cond_train=cond_train, cond_val=cond_train , patience=100 ,batch_size=8,n_epochs=1000)
 
  test_labels = np.random.choice(5, size=(400))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(400,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (400,1,28,28)) , test_labels



ways=5
shots=1
meta_lr=0.010
fast_lr=0.30
meta_batch_size=16
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )

# dim = 28*28

device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotCNN(ways)
# model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels  , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # # Compute meta-validation loss
        # batch = tasksets.validation.sample()
        
        # learner = maml.clone()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()


acc_acuracy=[]
iterations= 600
for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


215it [00:40,  5.25it/s, epoch_progress=100%, train_loss=-2.335e+05, last_val_loss=+1.560e+26, best_epoch=115, best_loss=+3.166e+25]
117it [00:22,  5.27it/s, epoch_progress=100%, train_loss=-1.871e+05, last_val_loss=+1.141e+26, best_epoch=17, best_loss=+2.943e+25]
166it [00:31,  5.23it/s, epoch_progress=100%, train_loss=-2.466e+05, last_val_loss=+1.752e+26, best_epoch=66, best_loss=+4.217e+25]
120it [00:22,  5.26it/s, epoch_progress=100%, train_loss=-2.693e+05, last_val_loss=+1.556e+26, best_epoch=20, best_loss=+3.539e+25]
174it [00:33,  5.25it/s, epoch_progress=100%, train_loss=-2.499e+05, last_val_loss=+1.242e+26, best_epoch=74, best_loss=+3.265e+25]
149it [00:28,  5.20it/s, epoch_progress=100%, train_loss=-2.901e+05, last_val_loss=+8.526e+25, best_epoch=49, best_loss=+3.676e+25]
101it [00:19,  5.29it/s, epoch_progress=100%, train_loss=-2.189e+05, last_val_loss=+1.919e+26, best_epoch=1, best_loss=+3.844e+05]
115it [00:21,  5.27it/s, epoch_progress=100%, train_loss=-2.129e+05, last_va



Iteration 0
Meta Train Error 1.9390601515769958
Meta Train Accuracy 0.18593750288709998


128it [00:24,  5.23it/s, epoch_progress=100%, train_loss=-2.293e+05, last_val_loss=+9.975e+25, best_epoch=28, best_loss=+4.189e+25]
132it [00:25,  5.26it/s, epoch_progress=100%, train_loss=-2.469e+05, last_val_loss=+3.050e+26, best_epoch=32, best_loss=+4.441e+25]
116it [00:21,  5.30it/s, epoch_progress=100%, train_loss=-2.674e+05, last_val_loss=+1.191e+26, best_epoch=16, best_loss=+4.734e+25]
117it [00:22,  5.19it/s, epoch_progress=100%, train_loss=-2.516e+05, last_val_loss=+1.825e+26, best_epoch=17, best_loss=+2.827e+25]
117it [00:22,  5.31it/s, epoch_progress=100%, train_loss=-2.810e+05, last_val_loss=+1.013e+26, best_epoch=17, best_loss=+4.942e+25]
114it [00:21,  5.21it/s, epoch_progress=100%, train_loss=-2.257e+05, last_val_loss=+7.212e+25, best_epoch=14, best_loss=+2.162e+25]
191it [00:36,  5.19it/s, epoch_progress=100%, train_loss=-2.508e+05, last_val_loss=+9.217e+25, best_epoch=91, best_loss=+2.987e+25]
116it [00:21,  5.34it/s, epoch_progress=100%, train_loss=-2.461e+05, last_va



Iteration 1
Meta Train Error 1.8743568360805511
Meta Train Accuracy 0.19687500270083547


116it [00:21,  5.38it/s, epoch_progress=100%, train_loss=-2.749e+05, last_val_loss=+7.601e+25, best_epoch=16, best_loss=+1.639e+25]
261it [00:48,  5.39it/s, epoch_progress=100%, train_loss=-2.158e+05, last_val_loss=+3.189e+26, best_epoch=161, best_loss=+4.252e+25]
119it [00:21,  5.43it/s, epoch_progress=100%, train_loss=-2.528e+05, last_val_loss=+9.600e+25, best_epoch=19, best_loss=+2.988e+25]
160it [00:29,  5.37it/s, epoch_progress=100%, train_loss=-2.794e+05, last_val_loss=+2.260e+26, best_epoch=60, best_loss=+3.039e+25]
121it [00:22,  5.36it/s, epoch_progress=100%, train_loss=-2.357e+05, last_val_loss=+1.877e+26, best_epoch=21, best_loss=+4.627e+25]
115it [00:21,  5.38it/s, epoch_progress=100%, train_loss=-2.641e+05, last_val_loss=+9.550e+25, best_epoch=15, best_loss=+2.526e+25]
131it [00:24,  5.42it/s, epoch_progress=100%, train_loss=-2.314e+05, last_val_loss=+2.724e+26, best_epoch=31, best_loss=+3.582e+25]
124it [00:22,  5.42it/s, epoch_progress=100%, train_loss=-2.167e+05, last_v



Iteration 2
Meta Train Error 1.736103892326355
Meta Train Accuracy 0.19375000288709998


Mean accuracy/loss, stddev, and confidence intervals
(0.6348125119693577, 0.0569568206496079, 0.004557494833450018)
0.450000012293458
0.8125000074505806


**SUMMARY OF EXP7**
---

lower fast_learning rate 0.35  and 3 iterations 

* **Standard MAML**

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.5572500119513522, 0.08597201673500991, 0.00687919405652049)

------
* **with the following flows**

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.6215000123189142, 0.08055950191919135, 0.0064461026720695125)


In [ ]:
#@title COMPARE WITH EXP7 standard maml with fast_lr 0.35 and 3 iterations and rest of parameters as EXP4

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.5572500119513522, 0.08597201673500991, 0.00687919405652049)

#@markdown min accuracy: 0.3000000063329935

#@markdown max accuracy: 0.8500000089406967
"""
  Demonstrates how to:
    * use the MAML wrapper for fast-adaptation,
    * use the benchmark interface to load Omniglot, and
    * sample tasks and split them in adaptation and evaluation sets.
"""

import random
import numpy as np
import torch
import learn2learn as l2l

from torch import nn, optim



def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


ways=5
shots=1
meta_lr=0.013
fast_lr=0.35
meta_batch_size=8
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42,

# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
device = torch.device('cpu')
if cuda:
    # torch.cuda.manual_seed(seed)
    device = torch.device('cuda')

# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                              train_ways=ways,
                                              train_samples=2*shots,
                                              test_ways=ways,
                                              test_samples=2*shots,
                                              num_tasks=20000,
                                              root='~/data',
)

# Create model
model =  l2l.vision.models.OmniglotCNN(ways)
#model =  l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')

for iteration in range(num_iterations):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-training loss
        learner = maml.clone()
        batch = tasksets.train.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

        # Compute meta-validation loss
        # learner = maml.clone()
        # batch = tasksets.validation.sample()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,
        #                                                     learner,
        #                                                     loss,
        #                                                     adaptation_steps,
        #                                                     shots,
        #                                                     ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()



acc_acuracy=[]
iterations=600

for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


Iteration 0
Meta Train Error 2.406614050269127
Meta Train Accuracy 0.2500000074505806


Iteration 1
Meta Train Error 1.4657954163849354
Meta Train Accuracy 0.42500000819563866


Iteration 2
Meta Train Error 1.4867252483963966
Meta Train Accuracy 0.42500000819563866
Mean accuracy/loss, stddev, and confidence intervals
(0.5572500119513522, 0.08597201673500991, 0.00687919405652049)
0.3000000063329935
0.8500000089406967


In [ ]:
#@title EXP7 lower fast_learning rate 0.35  and 3 iterations 
#@markdown meta_lr 0.013 

#@markdown 400 generated images instead of 1 shot per class 

#@markdown Foud that adding more layers of folws specifically batch normalization flow layer achives higher accuracy 

#@markdown Flow details:

#@markdown    BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),

#@markdown    BatchNorm(dim=dim), 6 times 

#@markdown    Affine(dim=dim),Affine(dim=dim),

#@markdown BatchNorm(dim=dim), 25 times 

#@markdown ------

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.6215000123189142, 0.08055950191919135, 0.0064461026720695125)

#@markdown min accuracy: 0.3750000074505806

#@markdown max accuracy: 0.9000000059604645


def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    indices = np.random.permutation(data.shape[0])
    
    adaptation_indices = indices[:360]
    evaluation_indices = indices[360:]
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # print("modified version")
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  # valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  # cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  
  flow = Sequential(      

      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),

  # very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# //////////////////
    #     BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# ////////
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    # MADE(DSF(dim=dim), cond_dim=cond_dim),
    Affine(dim=dim),  
    Affine(dim=dim),  
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),

      # BatchNorm(dim=dim),
      # Shuffle(dim=dim),
      # CouplingLayers(AffineTransformer(dim=dim//2 ) ,dim=dim),

  ).to(device)
  train_losses, val_losses = train(flow, trainX, trainX ,cond_train=cond_train, cond_val=cond_train , patience=100 ,batch_size=8,n_epochs=1000)
 
  test_labels = np.random.choice(5, size=(400))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(400,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (400,1,28,28)) , test_labels



ways=5
shots=1
meta_lr=0.013
fast_lr=0.35
meta_batch_size=8
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )

# dim = 28*28

device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotCNN(ways)
# model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels  , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # # Compute meta-validation loss
        # batch = tasksets.validation.sample()
        
        # learner = maml.clone()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()


acc_acuracy=[]
iterations= 600
for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


121it [00:19,  6.22it/s, epoch_progress=100%, train_loss=-2.157e+05, last_val_loss=+1.871e+22, best_epoch=21, best_loss=+7.114e+21]
116it [00:18,  6.20it/s, epoch_progress=100%, train_loss=-2.377e+05, last_val_loss=+2.909e+22, best_epoch=16, best_loss=+3.492e+21]
120it [00:19,  6.14it/s, epoch_progress=100%, train_loss=-2.272e+05, last_val_loss=+1.630e+22, best_epoch=20, best_loss=+7.546e+21]
152it [00:25,  6.08it/s, epoch_progress=100%, train_loss=-2.214e+05, last_val_loss=+4.263e+22, best_epoch=52, best_loss=+9.875e+21]
116it [00:19,  6.09it/s, epoch_progress=100%, train_loss=-1.822e+05, last_val_loss=+1.264e+22, best_epoch=16, best_loss=+5.174e+21]
120it [00:19,  6.09it/s, epoch_progress=100%, train_loss=-2.026e+05, last_val_loss=+1.741e+22, best_epoch=20, best_loss=+5.378e+21]
117it [00:18,  6.17it/s, epoch_progress=100%, train_loss=-2.104e+05, last_val_loss=+1.119e+22, best_epoch=17, best_loss=+4.739e+21]
118it [00:19,  6.07it/s, epoch_progress=100%, train_loss=-1.799e+05, last_va



Iteration 0
Meta Train Error 2.0482741594314575
Meta Train Accuracy 0.14375000214204192


169it [00:27,  6.22it/s, epoch_progress=100%, train_loss=-2.071e+05, last_val_loss=+1.371e+22, best_epoch=69, best_loss=+6.994e+21]
118it [00:18,  6.30it/s, epoch_progress=100%, train_loss=-1.911e+05, last_val_loss=+1.831e+22, best_epoch=18, best_loss=+4.323e+21]
119it [00:18,  6.33it/s, epoch_progress=100%, train_loss=-2.164e+05, last_val_loss=+1.394e+22, best_epoch=19, best_loss=+2.955e+21]
120it [00:19,  6.28it/s, epoch_progress=100%, train_loss=-1.819e+05, last_val_loss=+1.407e+22, best_epoch=20, best_loss=+4.070e+21]
119it [00:18,  6.32it/s, epoch_progress=100%, train_loss=-1.760e+05, last_val_loss=+9.325e+21, best_epoch=19, best_loss=+7.412e+21]
145it [00:23,  6.28it/s, epoch_progress=100%, train_loss=-2.408e+05, last_val_loss=+1.512e+22, best_epoch=45, best_loss=+5.344e+21]
117it [00:18,  6.45it/s, epoch_progress=100%, train_loss=-1.854e+05, last_val_loss=+1.674e+22, best_epoch=17, best_loss=+4.872e+21]
124it [00:19,  6.25it/s, epoch_progress=100%, train_loss=-2.105e+05, last_va



Iteration 1
Meta Train Error 1.7234332263469696
Meta Train Accuracy 0.23750000353902578


119it [00:18,  6.34it/s, epoch_progress=100%, train_loss=-1.963e+05, last_val_loss=+1.669e+22, best_epoch=19, best_loss=+6.158e+21]
122it [00:19,  6.18it/s, epoch_progress=100%, train_loss=-2.350e+05, last_val_loss=+1.627e+22, best_epoch=22, best_loss=+7.052e+21]
136it [00:21,  6.35it/s, epoch_progress=100%, train_loss=-1.865e+05, last_val_loss=+1.091e+22, best_epoch=36, best_loss=+5.672e+21]
121it [00:19,  6.29it/s, epoch_progress=100%, train_loss=-2.377e+05, last_val_loss=+9.967e+21, best_epoch=21, best_loss=+5.124e+21]
130it [00:21,  6.16it/s, epoch_progress=100%, train_loss=-2.071e+05, last_val_loss=+1.660e+22, best_epoch=30, best_loss=+5.688e+21]
118it [00:18,  6.23it/s, epoch_progress=100%, train_loss=-2.019e+05, last_val_loss=+6.038e+23, best_epoch=18, best_loss=+6.320e+21]
116it [00:18,  6.19it/s, epoch_progress=100%, train_loss=-1.871e+05, last_val_loss=+1.160e+22, best_epoch=16, best_loss=+9.125e+21]
120it [00:19,  6.13it/s, epoch_progress=100%, train_loss=-2.024e+05, last_va



Iteration 2
Meta Train Error 2.1149842590093613
Meta Train Accuracy 0.2187500037252903


Mean accuracy/loss, stddev, and confidence intervals
(0.6215000123189142, 0.08055950191919135, 0.0064461026720695125)
0.3750000074505806
0.9000000059604645



**SUMMARY OF EXP6**
---

lower fast_learning rate 0.3  and 3 iterations 

* **Standard MAML** 

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.5623750121705234, 0.08327905596336382, 0.006663712316783581)

----
* **with the following flows**

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.6194583455100655, 0.08578994689481148, 0.006864625435123306)


In [ ]:
#@title COMPARE WITH EXP6 standard maml with fast_lr 0.3 and 3 iterations and rest of parameters as EXP4

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.5623750121705234, 0.08327905596336382, 0.006663712316783581)

#@markdown min accuracy: 0.35000000335276127

#@markdown max accuracy: 0.8250000067055225

"""
  Demonstrates how to:
    * use the MAML wrapper for fast-adaptation,
    * use the benchmark interface to load Omniglot, and
    * sample tasks and split them in adaptation and evaluation sets.
"""

import random
import numpy as np
import torch
import learn2learn as l2l

from torch import nn, optim



def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


ways=5
shots=1
meta_lr=0.013
fast_lr=0.3
meta_batch_size=8
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42,

# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
device = torch.device('cpu')
if cuda:
    # torch.cuda.manual_seed(seed)
    device = torch.device('cuda')

# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                              train_ways=ways,
                                              train_samples=2*shots,
                                              test_ways=ways,
                                              test_samples=2*shots,
                                              num_tasks=20000,
                                              root='~/data',
)

# Create model
model =  l2l.vision.models.OmniglotCNN(ways)
#model =  l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')

for iteration in range(num_iterations):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-training loss
        learner = maml.clone()
        batch = tasksets.train.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

        # Compute meta-validation loss
        # learner = maml.clone()
        # batch = tasksets.validation.sample()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,
        #                                                     learner,
        #                                                     loss,
        #                                                     adaptation_steps,
        #                                                     shots,
        #                                                     ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()



acc_acuracy=[]
iterations=600

for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


Iteration 0
Meta Train Error 2.4023159593343735
Meta Train Accuracy 0.2750000059604645


Iteration 1
Meta Train Error 1.9544251561164856
Meta Train Accuracy 0.4000000078231096


Iteration 2
Meta Train Error 1.7129931673407555
Meta Train Accuracy 0.4500000085681677
Mean accuracy/loss, stddev, and confidence intervals
(0.5623750121705234, 0.08327905596336382, 0.006663712316783581)
0.35000000335276127
0.8250000067055225


In [ ]:
#@title EXP6 lower fast_learning rate 0.3  and 3 iterations 
#@markdown meta_lr 0.013 

#@markdown 400 generated images instead of 1 shot per class 

#@markdown Foud that adding more layers of folws specifically batch normalization flow layer achives higher accuracy 

#@markdown Flow details:

#@markdown    BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),

#@markdown    BatchNorm(dim=dim), 6 times 

#@markdown    Affine(dim=dim),Affine(dim=dim),


#@markdown BatchNorm(dim=dim), 25 times 

#@markdown -----

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.6194583455100655, 0.08578994689481148, 0.006864625435123306)

#@markdown min accuracy: 0.3000000063329935

#@markdown max accuracy: 0.8500000089406967

def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    indices = np.random.permutation(data.shape[0])
    
    adaptation_indices = indices[:360]
    evaluation_indices = indices[360:]
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # print("modified version")
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  # valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  # cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  
  flow = Sequential(      

      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),

  # very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# //////////////////
    #     BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# ////////
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    # MADE(DSF(dim=dim), cond_dim=cond_dim),
    Affine(dim=dim),  
    Affine(dim=dim),  
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),

      # BatchNorm(dim=dim),
      # Shuffle(dim=dim),
      # CouplingLayers(AffineTransformer(dim=dim//2 ) ,dim=dim),

  ).to(device)
  train_losses, val_losses = train(flow, trainX, trainX ,cond_train=cond_train, cond_val=cond_train , patience=100 ,batch_size=8,n_epochs=1000)
 
  test_labels = np.random.choice(5, size=(400))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(400,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (400,1,28,28)) , test_labels



ways=5
shots=1
meta_lr=0.013
fast_lr=0.3
meta_batch_size=8
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )

# dim = 28*28

device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotCNN(ways)
# model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels  , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # # Compute meta-validation loss
        # batch = tasksets.validation.sample()
        
        # learner = maml.clone()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()


acc_acuracy=[]
iterations= 600
for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


158it [00:25,  6.18it/s, epoch_progress=100%, train_loss=-2.131e+05, last_val_loss=+1.212e+22, best_epoch=58, best_loss=+7.539e+21]
116it [00:18,  6.17it/s, epoch_progress=100%, train_loss=-1.931e+05, last_val_loss=+1.498e+22, best_epoch=16, best_loss=+2.559e+21]
118it [00:18,  6.23it/s, epoch_progress=100%, train_loss=-2.231e+05, last_val_loss=+1.463e+22, best_epoch=18, best_loss=+1.260e+21]
134it [00:21,  6.28it/s, epoch_progress=100%, train_loss=-1.803e+05, last_val_loss=+1.213e+22, best_epoch=34, best_loss=+5.320e+21]
117it [00:18,  6.27it/s, epoch_progress=100%, train_loss=-2.431e+05, last_val_loss=+1.504e+22, best_epoch=17, best_loss=+2.121e+21]
151it [00:23,  6.35it/s, epoch_progress=100%, train_loss=-2.298e+05, last_val_loss=+1.885e+22, best_epoch=51, best_loss=+5.011e+21]
139it [00:22,  6.19it/s, epoch_progress=100%, train_loss=-2.464e+05, last_val_loss=+2.158e+22, best_epoch=39, best_loss=+6.262e+21]
116it [00:18,  6.30it/s, epoch_progress=100%, train_loss=-2.539e+05, last_va



Iteration 0
Meta Train Error 1.8237886130809784
Meta Train Accuracy 0.20000000298023224


121it [00:19,  6.24it/s, epoch_progress=100%, train_loss=-2.122e+05, last_val_loss=+3.005e+22, best_epoch=21, best_loss=+6.557e+21]
125it [00:19,  6.34it/s, epoch_progress=100%, train_loss=-2.117e+05, last_val_loss=+2.679e+22, best_epoch=25, best_loss=+6.031e+21]
116it [00:18,  6.31it/s, epoch_progress=100%, train_loss=-2.134e+05, last_val_loss=+2.246e+22, best_epoch=16, best_loss=+5.310e+21]
117it [00:19,  6.13it/s, epoch_progress=100%, train_loss=-2.252e+05, last_val_loss=+3.898e+23, best_epoch=17, best_loss=+3.590e+21]
239it [00:38,  6.23it/s, epoch_progress=100%, train_loss=-1.920e+05, last_val_loss=+2.237e+22, best_epoch=139, best_loss=+7.123e+21]
119it [00:19,  6.14it/s, epoch_progress=100%, train_loss=-2.269e+05, last_val_loss=+1.270e+22, best_epoch=19, best_loss=+3.217e+21]
122it [00:19,  6.14it/s, epoch_progress=100%, train_loss=-1.624e+05, last_val_loss=+7.836e+21, best_epoch=22, best_loss=+4.787e+21]
123it [00:19,  6.18it/s, epoch_progress=100%, train_loss=-2.515e+05, last_v



Iteration 1
Meta Train Error 1.7138043642044067
Meta Train Accuracy 0.2343750074505806


114it [00:18,  6.19it/s, epoch_progress=100%, train_loss=-2.267e+05, last_val_loss=+1.098e+22, best_epoch=14, best_loss=+3.678e+21]
128it [00:21,  6.02it/s, epoch_progress=100%, train_loss=-1.883e+05, last_val_loss=+1.772e+28, best_epoch=28, best_loss=+7.278e+21]
154it [00:25,  6.08it/s, epoch_progress=100%, train_loss=-1.781e+05, last_val_loss=+2.336e+22, best_epoch=54, best_loss=+7.276e+21]
179it [00:28,  6.18it/s, epoch_progress=100%, train_loss=-1.907e+05, last_val_loss=+3.013e+22, best_epoch=79, best_loss=+8.359e+21]
121it [00:19,  6.12it/s, epoch_progress=100%, train_loss=-1.950e+05, last_val_loss=+2.129e+22, best_epoch=21, best_loss=+5.357e+21]
125it [00:20,  6.04it/s, epoch_progress=100%, train_loss=-2.314e+05, last_val_loss=+1.753e+22, best_epoch=25, best_loss=+3.301e+21]
116it [00:18,  6.11it/s, epoch_progress=100%, train_loss=-2.342e+05, last_val_loss=+1.532e+22, best_epoch=16, best_loss=+3.329e+21]
132it [00:21,  6.28it/s, epoch_progress=100%, train_loss=-2.086e+05, last_va



Iteration 2
Meta Train Error 1.7486406862735748
Meta Train Accuracy 0.17187500279396772


Mean accuracy/loss, stddev, and confidence intervals
(0.6194583455100655, 0.08578994689481148, 0.006864625435123306)
0.3000000063329935
0.8500000089406967



**SUMMARY OF EXP5**
---

lower fast_learning rate 0.2  and 3 iterations 

* **Standard MAML** 

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.5799166787912449, 0.08488223104448211, 0.006791993040075019)

-----

* **with the following flows**

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.6055000121239573, 0.08472357040113233, 0.006779297544538376)



In [ ]:
#@title COMPARE WITH EXP5 standard maml with fast_lr 0.2 and 3 iterations and rest of parameters as EXP4

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.5799166787912449, 0.08488223104448211, 0.006791993040075019)

#@markdown 0.32500000670552254

#@markdown 0.8250000067055225

"""
  Demonstrates how to:
    * use the MAML wrapper for fast-adaptation,
    * use the benchmark interface to load Omniglot, and
    * sample tasks and split them in adaptation and evaluation sets.
"""

import random
import numpy as np
import torch
import learn2learn as l2l

from torch import nn, optim



def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


ways=5
shots=1
meta_lr=0.013
fast_lr=0.2
meta_batch_size=8
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42,

# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
device = torch.device('cpu')
if cuda:
    # torch.cuda.manual_seed(seed)
    device = torch.device('cuda')

# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                              train_ways=ways,
                                              train_samples=2*shots,
                                              test_ways=ways,
                                              test_samples=2*shots,
                                              num_tasks=20000,
                                              root='~/data',
)

# Create model
model =  l2l.vision.models.OmniglotCNN(ways)
#model =  l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')

for iteration in range(num_iterations):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-training loss
        learner = maml.clone()
        batch = tasksets.train.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

        # Compute meta-validation loss
        # learner = maml.clone()
        # batch = tasksets.validation.sample()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,
        #                                                     learner,
        #                                                     loss,
        #                                                     adaptation_steps,
        #                                                     shots,
        #                                                     ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()



acc_acuracy=[]
iterations=600

for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


Iteration 0
Meta Train Error 1.3961520940065384
Meta Train Accuracy 0.4500000085681677


Iteration 1
Meta Train Error 1.56709473580122
Meta Train Accuracy 0.4250000063329935


Iteration 2
Meta Train Error 1.0159304700791836
Meta Train Accuracy 0.7250000108033419
Mean accuracy/loss, stddev, and confidence intervals
(0.5799166787912449, 0.08488223104448211, 0.006791993040075019)
0.32500000670552254
0.8250000067055225


In [ ]:
#@title EXP5 lower fast_learning rate 0.2  and 3 iterations 
#@markdown meta_lr 0.013 

#@markdown 400 generated images instead of 1 shot per class 

#@markdown Foud that adding more layers of folws specifically batch normalization flow layer achives higher accuracy 

#@markdown Flow details:

#@markdown    BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),

#@markdown    BatchNorm(dim=dim), 6 times 

#@markdown    Affine(dim=dim),Affine(dim=dim),


#@markdown BatchNorm(dim=dim), 25 times 

#@markdown ----

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.6055000121239573, 0.08472357040113233, 0.006779297544538376)

#@markdown min accuracy: 0.3500000052154064

#@markdown max accuracy: 0.8500000089406967

def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    indices = np.random.permutation(data.shape[0])
    
    adaptation_indices = indices[:360]
    evaluation_indices = indices[360:]
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # print("modified version")
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  # valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  # cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  
  flow = Sequential(      

      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),

  # very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# //////////////////
    #     BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# ////////
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    # MADE(DSF(dim=dim), cond_dim=cond_dim),
    Affine(dim=dim),  
    Affine(dim=dim),  
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),

      # BatchNorm(dim=dim),
      # Shuffle(dim=dim),
      # CouplingLayers(AffineTransformer(dim=dim//2 ) ,dim=dim),

  ).to(device)
  train_losses, val_losses = train(flow, trainX, trainX ,cond_train=cond_train, cond_val=cond_train , patience=100 ,batch_size=8,n_epochs=1000)
 
  test_labels = np.random.choice(5, size=(400))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(400,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (400,1,28,28)) , test_labels



ways=5
shots=1
meta_lr=0.013
fast_lr=0.2
meta_batch_size=8
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )

# dim = 28*28

device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotCNN(ways)
# model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels  , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # # Compute meta-validation loss
        # batch = tasksets.validation.sample()
        
        # learner = maml.clone()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()


acc_acuracy=[]
iterations= 600
for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


166it [00:26,  6.18it/s, epoch_progress=100%, train_loss=-1.778e+05, last_val_loss=+1.554e+22, best_epoch=66, best_loss=+8.660e+21]
142it [00:22,  6.23it/s, epoch_progress=100%, train_loss=-2.013e+05, last_val_loss=+6.145e+21, best_epoch=42, best_loss=+4.804e+21]
115it [00:18,  6.23it/s, epoch_progress=100%, train_loss=-2.254e+05, last_val_loss=+5.582e+21, best_epoch=15, best_loss=+3.139e+21]
155it [00:25,  6.19it/s, epoch_progress=100%, train_loss=-1.752e+05, last_val_loss=+2.089e+22, best_epoch=55, best_loss=+6.566e+21]
122it [00:19,  6.25it/s, epoch_progress=100%, train_loss=-1.914e+05, last_val_loss=+1.621e+22, best_epoch=22, best_loss=+5.538e+21]
118it [00:19,  6.09it/s, epoch_progress=100%, train_loss=-1.979e+05, last_val_loss=+2.142e+22, best_epoch=18, best_loss=+5.643e+21]
118it [00:19,  6.04it/s, epoch_progress=100%, train_loss=-2.315e+05, last_val_loss=+1.948e+22, best_epoch=18, best_loss=+2.097e+21]
124it [00:20,  6.09it/s, epoch_progress=100%, train_loss=-2.474e+05, last_va



Iteration 0
Meta Train Error 1.9839521050453186
Meta Train Accuracy 0.17812499962747097


124it [00:19,  6.22it/s, epoch_progress=100%, train_loss=-1.551e+05, last_val_loss=+1.056e+22, best_epoch=24, best_loss=+5.629e+21]
123it [00:19,  6.18it/s, epoch_progress=100%, train_loss=-2.089e+05, last_val_loss=+1.023e+22, best_epoch=23, best_loss=+4.600e+21]
117it [00:18,  6.19it/s, epoch_progress=100%, train_loss=-1.967e+05, last_val_loss=+1.689e+22, best_epoch=17, best_loss=+3.069e+21]
115it [00:18,  6.31it/s, epoch_progress=100%, train_loss=-2.152e+05, last_val_loss=+2.794e+22, best_epoch=15, best_loss=+7.502e+21]
152it [00:24,  6.21it/s, epoch_progress=100%, train_loss=-2.080e+05, last_val_loss=+2.277e+22, best_epoch=52, best_loss=+9.384e+21]
137it [00:21,  6.25it/s, epoch_progress=100%, train_loss=-2.103e+05, last_val_loss=+2.813e+22, best_epoch=37, best_loss=+5.945e+21]
117it [00:19,  6.15it/s, epoch_progress=100%, train_loss=-2.113e+05, last_val_loss=+1.182e+22, best_epoch=17, best_loss=+3.789e+21]
114it [00:18,  6.19it/s, epoch_progress=100%, train_loss=-2.289e+05, last_va



Iteration 1
Meta Train Error 1.764745444059372
Meta Train Accuracy 0.17500000353902578


114it [00:18,  6.27it/s, epoch_progress=100%, train_loss=-2.096e+05, last_val_loss=+1.178e+22, best_epoch=14, best_loss=+2.064e+21]
124it [00:20,  6.16it/s, epoch_progress=100%, train_loss=-2.271e+05, last_val_loss=+1.216e+22, best_epoch=24, best_loss=+3.808e+21]
150it [00:23,  6.27it/s, epoch_progress=100%, train_loss=-2.162e+05, last_val_loss=+2.012e+22, best_epoch=50, best_loss=+2.666e+21]
118it [00:19,  6.14it/s, epoch_progress=100%, train_loss=-1.743e+05, last_val_loss=+1.198e+22, best_epoch=18, best_loss=+5.975e+21]
114it [00:18,  6.25it/s, epoch_progress=100%, train_loss=-2.056e+05, last_val_loss=+3.300e+22, best_epoch=14, best_loss=+1.244e+21]
121it [00:19,  6.10it/s, epoch_progress=100%, train_loss=-1.974e+05, last_val_loss=+1.688e+22, best_epoch=21, best_loss=+6.867e+21]
116it [00:19,  6.11it/s, epoch_progress=100%, train_loss=-1.963e+05, last_val_loss=+8.433e+23, best_epoch=16, best_loss=+5.754e+21]
133it [00:21,  6.06it/s, epoch_progress=100%, train_loss=-2.053e+05, last_va



Iteration 2
Meta Train Error 1.6917263120412827
Meta Train Accuracy 0.1906250063329935


Mean accuracy/loss, stddev, and confidence intervals
(0.6055000121239573, 0.08472357040113233, 0.006779297544538376)
0.3500000052154064
0.8500000089406967


**SUMMARY OF EXP4**
----

higher meta learning rate 0.013  and 3 iterations 

* **Standard MAML** 

  *  Mean accuracy/loss, stddev, and confidence intervals
  * (0.5756250124828269, 0.08111607608978898, 0.006490637880987425)

-----

* **with the following flows**

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.5979166786403706, 0.08530480494926775, 0.006825805994622801)



In [ ]:
#@title COMPARE WITH EXP4 Standart MAML with meta lr 0.013 on 3 iterations 

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.5756250124828269, 0.08111607608978898, 0.006490637880987425)

#@markdown min accuracy: 0.3000000100582838

#@markdown max accuracy: 0.7750000134110451

"""
  Demonstrates how to:
    * use the MAML wrapper for fast-adaptation,
    * use the benchmark interface to load Omniglot, and
    * sample tasks and split them in adaptation and evaluation sets.
"""

import random
import numpy as np
import torch
import learn2learn as l2l

from torch import nn, optim



def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


ways=5
shots=1
meta_lr=0.013
fast_lr=0.5
meta_batch_size=8
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42,

# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
device = torch.device('cpu')
if cuda:
    # torch.cuda.manual_seed(seed)
    device = torch.device('cuda')

# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                              train_ways=ways,
                                              train_samples=2*shots,
                                              test_ways=ways,
                                              test_samples=2*shots,
                                              num_tasks=20000,
                                              root='~/data',
)

# Create model
model =  l2l.vision.models.OmniglotCNN(ways)
#model =  l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')

for iteration in range(num_iterations):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-training loss
        learner = maml.clone()
        batch = tasksets.train.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

        # Compute meta-validation loss
        # learner = maml.clone()
        # batch = tasksets.validation.sample()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,
        #                                                     learner,
        #                                                     loss,
        #                                                     adaptation_steps,
        #                                                     shots,
        #                                                     ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()



acc_acuracy=[]
iterations=600

for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


Iteration 0
Meta Train Error 1.9204799234867096
Meta Train Accuracy 0.3000000063329935


Iteration 1
Meta Train Error 1.7800371944904327
Meta Train Accuracy 0.42500001192092896


Iteration 2
Meta Train Error 1.650674894452095
Meta Train Accuracy 0.4250000100582838
Mean accuracy/loss, stddev, and confidence intervals
(0.5756250124828269, 0.08111607608978898, 0.006490637880987425)
0.3000000100582838
0.7750000134110451


In [ ]:
#@title EXP4 with normalizing flow and higer meta_lr 0.013 on 3 on 3 iterations

#@markdown 400 generated images instead of 1 shot per class 

#@markdown Foud that adding more layers of folws specifically batch normalization flow layer achives higher accuracy 

#@markdown Flow details:

#@markdown    BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),BatchNorm(dim=dim),ActNorm(dim=dim).warm_start(x=trainX),

#@markdown    BatchNorm(dim=dim), 6 times 

#@markdown    Affine(dim=dim),Affine(dim=dim),


#@markdown BatchNorm(dim=dim), 25 times 

#@markdown -----

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.5979166786403706, 0.08530480494926775, 0.006825805994622801)

#@markdown min accuracy: 0.3500000089406967

#@markdown max accuracy: 0.8500000089406967 

def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    indices = np.random.permutation(data.shape[0])
    
    adaptation_indices = indices[:360]
    evaluation_indices = indices[360:]
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # print("modified version")
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  # valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  # cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  
  flow = Sequential(      

      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),

  # very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# //////////////////
    #     BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
    # BatchNorm(dim=dim),
    # ActNorm(dim=dim).warm_start(x=trainX),
# ////////
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    # MADE(DSF(dim=dim), cond_dim=cond_dim),
    Affine(dim=dim),  
    Affine(dim=dim),  
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),

      # BatchNorm(dim=dim),
      # Shuffle(dim=dim),
      # CouplingLayers(AffineTransformer(dim=dim//2 ) ,dim=dim),

  ).to(device)
  train_losses, val_losses = train(flow, trainX, trainX ,cond_train=cond_train, cond_val=cond_train , patience=100 ,batch_size=8,n_epochs=1000)
 
  test_labels = np.random.choice(5, size=(400))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(400,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (400,1,28,28)) , test_labels



ways=5
shots=1
meta_lr=0.013
fast_lr=0.5
meta_batch_size=8
adaptation_steps=1
num_iterations=3
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )

# dim = 28*28

device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotCNN(ways)
# model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels  , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # # Compute meta-validation loss
        # batch = tasksets.validation.sample()
        
        # learner = maml.clone()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()
/

acc_acuracy=[]
iterations= 600
for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


141it [00:21,  6.71it/s, epoch_progress=100%, train_loss=-2.216e+05, last_val_loss=+2.024e+22, best_epoch=41, best_loss=+8.101e+21]
116it [00:17,  6.72it/s, epoch_progress=100%, train_loss=-2.301e+05, last_val_loss=+1.284e+22, best_epoch=16, best_loss=+1.524e+21]
126it [00:18,  6.65it/s, epoch_progress=100%, train_loss=-1.963e+05, last_val_loss=+2.425e+22, best_epoch=26, best_loss=+2.654e+21]
126it [00:18,  6.82it/s, epoch_progress=100%, train_loss=-1.966e+05, last_val_loss=+6.942e+21, best_epoch=26, best_loss=+4.960e+21]
134it [00:19,  6.71it/s, epoch_progress=100%, train_loss=-2.058e+05, last_val_loss=+1.807e+22, best_epoch=34, best_loss=+4.316e+21]
149it [00:21,  6.80it/s, epoch_progress=100%, train_loss=-2.392e+05, last_val_loss=+1.905e+22, best_epoch=49, best_loss=+5.043e+21]
213it [00:31,  6.74it/s, epoch_progress=100%, train_loss=-2.645e+05, last_val_loss=+1.273e+22, best_epoch=113, best_loss=+4.650e+21]
166it [00:24,  6.66it/s, epoch_progress=100%, train_loss=-1.780e+05, last_v



Iteration 0
Meta Train Error 2.7952945232391357
Meta Train Accuracy 0.1843750039115548


120it [00:17,  6.73it/s, epoch_progress=100%, train_loss=-2.028e+05, last_val_loss=+9.757e+21, best_epoch=20, best_loss=+1.833e+21]
128it [00:19,  6.71it/s, epoch_progress=100%, train_loss=-2.087e+05, last_val_loss=+8.668e+21, best_epoch=28, best_loss=+4.499e+21]
132it [00:19,  6.70it/s, epoch_progress=100%, train_loss=-1.931e+05, last_val_loss=+2.087e+22, best_epoch=32, best_loss=+8.881e+21]
166it [00:24,  6.67it/s, epoch_progress=100%, train_loss=-2.276e+05, last_val_loss=+1.318e+22, best_epoch=66, best_loss=+3.994e+21]
101it [00:14,  6.83it/s, epoch_progress=100%, train_loss=-1.910e+05, last_val_loss=+1.430e+22, best_epoch=1, best_loss=+2.108e+04]
126it [00:18,  6.73it/s, epoch_progress=100%, train_loss=-2.009e+05, last_val_loss=+1.346e+22, best_epoch=26, best_loss=+7.908e+21]
164it [00:24,  6.74it/s, epoch_progress=100%, train_loss=-2.088e+05, last_val_loss=+1.310e+22, best_epoch=64, best_loss=+6.454e+21]
116it [00:17,  6.60it/s, epoch_progress=100%, train_loss=-2.431e+05, last_val



Iteration 1
Meta Train Error 2.34393247961998
Meta Train Accuracy 0.18437500344589353


119it [00:17,  6.64it/s, epoch_progress=100%, train_loss=-2.159e+05, last_val_loss=+1.051e+22, best_epoch=19, best_loss=+2.940e+21]
122it [00:18,  6.65it/s, epoch_progress=100%, train_loss=-2.094e+05, last_val_loss=+1.142e+22, best_epoch=22, best_loss=+6.362e+21]
183it [00:27,  6.73it/s, epoch_progress=100%, train_loss=-2.089e+05, last_val_loss=+1.141e+22, best_epoch=83, best_loss=+6.276e+21]
115it [00:16,  6.77it/s, epoch_progress=100%, train_loss=-1.908e+05, last_val_loss=+1.344e+22, best_epoch=15, best_loss=+4.190e+21]
139it [00:20,  6.63it/s, epoch_progress=100%, train_loss=-2.279e+05, last_val_loss=+3.192e+22, best_epoch=39, best_loss=+4.113e+21]
118it [00:17,  6.74it/s, epoch_progress=100%, train_loss=-2.240e+05, last_val_loss=+1.913e+22, best_epoch=18, best_loss=+5.594e+21]
147it [00:22,  6.64it/s, epoch_progress=100%, train_loss=-2.004e+05, last_val_loss=+1.702e+22, best_epoch=47, best_loss=+5.641e+21]
172it [00:25,  6.70it/s, epoch_progress=100%, train_loss=-1.911e+05, last_va



Iteration 2
Meta Train Error 2.3716704547405243
Meta Train Accuracy 0.22812500828877091


Mean accuracy/loss, stddev, and confidence intervals
(0.5979166786403706, 0.08530480494926775, 0.006825805994622801)
0.3500000089406967
0.8500000089406967


**Summary of EXP3**
---
Foud that adding more layers of folws specifically batch normalization flow layer achives higher accuracy

* **Standard MAML**
  * not tested 
----
* **with following flow**
  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.6064166788539539, 0.07988581283694113, 0.006392196318507478)



In [ ]:
#@title EXP3 400 generated images instead of 1 shot per class and 15 iterations

#@markdown Foud that adding more layers of folws specifically batch normalization flow layer achives higher accuracy 

#@markdown Flow details: 

#@markdown BatchNorm(dim=dim),

#@markdown ActNorm(dim=dim).warm_start(x=trainX),

#@markdown    BatchNorm(dim=dim),

#@markdown    ActNorm(dim=dim).warm_start(x=trainX),

#@markdown    BatchNorm(dim=dim),

#@markdown    ActNorm(dim=dim).warm_start(x=trainX),

#@markdown    BatchNorm(dim=dim),

#@markdown    BatchNorm(dim=dim),

#@markdown    BatchNorm(dim=dim),

#@markdown    BatchNorm(dim=dim),

#@markdown   BatchNorm(dim=dim),

#@markdown    BatchNorm(dim=dim),

#@markdown -----

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.6064166788539539, 0.07988581283694113, 0.006392196318507478)

#@markdown 0.35000001080334187

#@markdown 0.8250000104308128


def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    indices = np.random.permutation(data.shape[0])
    
    adaptation_indices = indices[:360]
    evaluation_indices = indices[360:]
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # print("modified version")
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  # valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  # cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  flow = Sequential(      

  # very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    BatchNorm(dim=dim),
    ActNorm(dim=dim).warm_start(x=trainX),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    BatchNorm(dim=dim),
    # MADE(DSF(dim=dim), cond_dim=cond_dim),
    # Affine(dim=dim),  

      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
      # BatchNorm(dim=dim),
      # Shuffle(dim=dim),
      # CouplingLayers(AffineTransformer(dim=dim//2 ) ,dim=dim),

  ).to(device)

  train_losses, val_losses = train(flow, trainX, trainX ,cond_train=cond_train, cond_val=cond_train , patience=100 ,batch_size=8,n_epochs=1000)
 
  test_labels = np.random.choice(5, size=(400))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(400,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (400,1,28,28)) , test_labels



ways=5
shots=1
meta_lr=0.003
fast_lr=0.5
meta_batch_size=8
adaptation_steps=1
num_iterations=15
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )

# dim = 28*28

device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotCNN(ways)
# model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels  , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # # Compute meta-validation loss
        # batch = tasksets.validation.sample()
        
        # learner = maml.clone()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()


acc_acuracy=[]
iterations= 600
for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


118it [00:06, 17.41it/s, epoch_progress=100%, train_loss=-6.552e+04, last_val_loss=+1.866e+07, best_epoch=18, best_loss=+1.073e+07]
131it [00:07, 17.93it/s, epoch_progress=100%, train_loss=-6.858e+04, last_val_loss=+3.157e+07, best_epoch=31, best_loss=+1.636e+07]
135it [00:07, 17.59it/s, epoch_progress=100%, train_loss=-7.263e+04, last_val_loss=+3.050e+07, best_epoch=35, best_loss=+1.851e+07]
112it [00:06, 18.01it/s, epoch_progress=100%, train_loss=-7.016e+04, last_val_loss=+1.378e+07, best_epoch=12, best_loss=+6.496e+06]
112it [00:06, 17.54it/s, epoch_progress=100%, train_loss=-6.565e+04, last_val_loss=+2.255e+07, best_epoch=12, best_loss=+9.927e+06]
124it [00:07, 17.58it/s, epoch_progress=100%, train_loss=-6.397e+04, last_val_loss=+2.673e+07, best_epoch=24, best_loss=+1.273e+07]
101it [00:05, 18.05it/s, epoch_progress=100%, train_loss=-6.936e+04, last_val_loss=+2.289e+07, best_epoch=1, best_loss=-5.324e+04]
113it [00:06, 17.88it/s, epoch_progress=100%, train_loss=-7.355e+04, last_val



Iteration 0
Meta Train Error 2.974050909280777
Meta Train Accuracy 0.2031250037252903


110it [00:06, 17.99it/s, epoch_progress=100%, train_loss=-7.711e+04, last_val_loss=+2.412e+07, best_epoch=10, best_loss=+4.353e+06]
111it [00:06, 17.45it/s, epoch_progress=100%, train_loss=-6.294e+04, last_val_loss=+2.575e+07, best_epoch=11, best_loss=+7.095e+06]
152it [00:08, 17.68it/s, epoch_progress=100%, train_loss=-6.770e+04, last_val_loss=+3.735e+07, best_epoch=52, best_loss=+1.498e+07]
113it [00:06, 17.47it/s, epoch_progress=100%, train_loss=-5.992e+04, last_val_loss=+2.218e+07, best_epoch=13, best_loss=+8.122e+06]
117it [00:06, 17.82it/s, epoch_progress=100%, train_loss=-6.328e+04, last_val_loss=+2.108e+07, best_epoch=17, best_loss=+9.954e+06]
110it [00:06, 17.79it/s, epoch_progress=100%, train_loss=-6.852e+04, last_val_loss=+3.429e+07, best_epoch=10, best_loss=+5.033e+06]
115it [00:06, 17.10it/s, epoch_progress=100%, train_loss=-8.158e+04, last_val_loss=+1.775e+07, best_epoch=15, best_loss=+9.203e+06]
117it [00:06, 17.14it/s, epoch_progress=100%, train_loss=-5.941e+04, last_va

...



Iteration 12
Meta Train Error 1.8904107362031937
Meta Train Accuracy 0.18750000186264515


115it [00:06, 17.30it/s, epoch_progress=100%, train_loss=-6.858e+04, last_val_loss=+2.847e+07, best_epoch=15, best_loss=+1.053e+07]
114it [00:06, 16.94it/s, epoch_progress=100%, train_loss=-7.920e+04, last_val_loss=+4.180e+07, best_epoch=14, best_loss=+1.604e+07]
113it [00:06, 17.24it/s, epoch_progress=100%, train_loss=-7.684e+04, last_val_loss=+2.667e+07, best_epoch=13, best_loss=+9.784e+06]
132it [00:07, 17.05it/s, epoch_progress=100%, train_loss=-7.099e+04, last_val_loss=+2.067e+07, best_epoch=32, best_loss=+1.328e+07]
111it [00:06, 17.07it/s, epoch_progress=100%, train_loss=-7.748e+04, last_val_loss=+1.828e+07, best_epoch=11, best_loss=+5.350e+06]
115it [00:06, 16.91it/s, epoch_progress=100%, train_loss=-7.077e+04, last_val_loss=+1.876e+07, best_epoch=15, best_loss=+1.147e+07]
110it [00:06, 17.12it/s, epoch_progress=100%, train_loss=-7.290e+04, last_val_loss=+1.761e+07, best_epoch=10, best_loss=+4.434e+06]
153it [00:08, 17.08it/s, epoch_progress=100%, train_loss=-5.063e+04, last_va



Iteration 13
Meta Train Error 1.8283534944057465
Meta Train Accuracy 0.21250000409781933


117it [00:06, 17.43it/s, epoch_progress=100%, train_loss=-7.413e+04, last_val_loss=+1.555e+07, best_epoch=17, best_loss=+9.279e+06]
112it [00:06, 17.07it/s, epoch_progress=100%, train_loss=-7.211e+04, last_val_loss=+2.102e+07, best_epoch=12, best_loss=+1.281e+07]
150it [00:08, 16.88it/s, epoch_progress=100%, train_loss=-7.254e+04, last_val_loss=+2.289e+07, best_epoch=50, best_loss=+1.382e+07]
126it [00:07, 16.99it/s, epoch_progress=100%, train_loss=-7.135e+04, last_val_loss=+2.740e+07, best_epoch=26, best_loss=+1.581e+07]
118it [00:06, 17.22it/s, epoch_progress=100%, train_loss=-6.397e+04, last_val_loss=+3.198e+07, best_epoch=18, best_loss=+1.415e+07]
116it [00:06, 17.24it/s, epoch_progress=100%, train_loss=-7.323e+04, last_val_loss=+1.493e+07, best_epoch=16, best_loss=+7.414e+06]
142it [00:08, 16.85it/s, epoch_progress=100%, train_loss=-6.830e+04, last_val_loss=+2.660e+07, best_epoch=42, best_loss=+1.885e+07]
134it [00:07, 17.03it/s, epoch_progress=100%, train_loss=-7.300e+04, last_va



Iteration 14
Meta Train Error 2.0196507275104523
Meta Train Accuracy 0.18750000558793545
Mean accuracy/loss, stddev, and confidence intervals
(0.5629583456149946, 0.08016778296272273, 0.006414758627582673)
0.32500000670552254
0.7500000149011612



**Summary of EXP2** 
---

400 generated images instead of 1 shot per class and 15 iterations, Flow of single batch norm layer

* **Standard MAML**
  * not tested 

---
* **with following flow**

  * Mean accuracy/loss, stddev, and confidence intervals
  * (0.5512083456323793, 0.07986706701169034, 0.006390696340089533)


In [ ]:
#@title EXP2 400 generated images instead of 1 shot per class and 15 iterations, Flow of single batch norm layer with pacience=100

#@markdown ----

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.5512083456323793, 0.07986706701169034, 0.006390696340089533)

#@markdown min accuracy: 0.2750000078231096

#@markdown max accuracy:  0.800000011920929



def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    indices = np.random.permutation(data.shape[0])
    
    adaptation_indices = indices[:360]
    evaluation_indices = indices[360:]
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # print("modified version")
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  # valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  # cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  flow = Sequential(      

# very slow in sampling 
    inv_flow(Sigmoid)(dim=dim),
    BatchNorm(dim=dim),
    # MADE(DSF(dim=dim), cond_dim=cond_dim),
    # Affine(dim=dim),  

      # BatchNorm(dim=dim),
      # MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
  ).to(device)


  train_losses, val_losses = train(flow, trainX, trainX ,cond_train=cond_train, cond_val=cond_train , patience=100 ,batch_size=8,n_epochs=1000)
 
  test_labels = np.random.choice(5, size=(400))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(400,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (400,1,28,28)) , test_labels



ways=5
shots=1
meta_lr=0.003
fast_lr=0.5
meta_batch_size=8
adaptation_steps=1
num_iterations=15
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )



device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotCNN(ways)
# model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels  , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # # Compute meta-validation loss
        # batch = tasksets.validation.sample()
        
        # learner = maml.clone()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()


acc_acuracy=[]
iterations= 600
for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


  0%|          | 0/9464212 [00:00<?, ?it/s]

Extracting /root/data/omniglot-py/images_background.zip to /root/data/omniglot-py


  0%|          | 0/6462886 [00:00<?, ?it/s]

Extracting /root/data/omniglot-py/images_evaluation.zip to /root/data/omniglot-py


...



Iteration 12
Meta Train Error 2.052701860666275
Meta Train Accuracy 0.1781250024214387


260it [00:06, 42.06it/s, epoch_progress=100%, train_loss=-9.997e+03, last_val_loss=-5.867e+03, best_epoch=160, best_loss=-5.912e+03]
1000it [00:23, 42.85it/s, epoch_progress=100%, train_loss=-8.002e+03, last_val_loss=-6.105e+03, best_epoch=977, best_loss=-6.150e+03]
1000it [00:23, 42.65it/s, epoch_progress=100%, train_loss=-9.803e+03, last_val_loss=-7.151e+03, best_epoch=995, best_loss=-7.207e+03]
1000it [00:23, 42.45it/s, epoch_progress=100%, train_loss=-1.062e+04, last_val_loss=-8.488e+03, best_epoch=994, best_loss=-8.502e+03]
103it [00:02, 43.39it/s, epoch_progress=100%, train_loss=-7.789e+03, last_val_loss=-5.637e+03, best_epoch=3, best_loss=-5.697e+03]
1000it [00:23, 42.48it/s, epoch_progress=100%, train_loss=-1.024e+04, last_val_loss=-7.910e+03, best_epoch=999, best_loss=-7.913e+03]
1000it [00:23, 42.73it/s, epoch_progress=100%, train_loss=-9.386e+03, last_val_loss=-6.915e+03, best_epoch=989, best_loss=-6.937e+03]
1000it [00:23, 42.55it/s, epoch_progress=100%, train_loss=-9.123e+



Iteration 13
Meta Train Error 1.97226083278656
Meta Train Accuracy 0.17812500335276127


1000it [00:23, 43.16it/s, epoch_progress=100%, train_loss=-1.006e+04, last_val_loss=-6.614e+03, best_epoch=998, best_loss=-6.651e+03]
1000it [00:23, 42.89it/s, epoch_progress=100%, train_loss=-1.023e+04, last_val_loss=-7.981e+03, best_epoch=998, best_loss=-8.022e+03]
1000it [00:23, 42.42it/s, epoch_progress=100%, train_loss=-8.657e+03, last_val_loss=-7.054e+03, best_epoch=1000, best_loss=-7.054e+03]
1000it [00:23, 42.86it/s, epoch_progress=100%, train_loss=-1.078e+04, last_val_loss=-8.029e+03, best_epoch=992, best_loss=-8.059e+03]
1000it [00:23, 42.19it/s, epoch_progress=100%, train_loss=-1.008e+04, last_val_loss=-6.420e+03, best_epoch=988, best_loss=-6.453e+03]
1000it [00:23, 42.43it/s, epoch_progress=100%, train_loss=-1.160e+04, last_val_loss=-8.741e+03, best_epoch=999, best_loss=-8.747e+03]
1000it [00:23, 42.07it/s, epoch_progress=100%, train_loss=-1.080e+04, last_val_loss=-7.353e+03, best_epoch=988, best_loss=-7.396e+03]
106it [00:02, 41.20it/s, epoch_progress=100%, train_loss=-9.5



Iteration 14
Meta Train Error 2.00375896692276
Meta Train Accuracy 0.20000000623986125
Mean accuracy/loss, stddev, and confidence intervals
(0.5512083456323793, 0.07986706701169034, 0.006390696340089533)
0.2750000078231096
0.800000011920929


**Summary of EXP 1**
---

original code from stage 1 with 400 generated images instead of 1 shot per class and 15 iterations
Flow of single batch norm layer and single MADE layer

* **Standard MAML**
  * not tested
---
* **with following flow**

  * Mean accuracy/loss, stddev, and confidence intervals
  *(0.5434166772446285, 0.10080092526546676, 0.008065753861693276)



In [ ]:
#@title EXP 1 original code from stage 1 with 400 generated images instead of 1 shot per class and 15 iterations  

#@markdown Flow of single batch norm layer and single MADE layer

#@markdown -----

#@markdown Mean accuracy/loss, stddev, and confidence intervals

#@markdown (0.5434166772446285, 0.10080092526546676, 0.008065753861693276)

#@markdown min accuracy: 0.27500000409781933

#@markdown max accuracy: 0.8250000067055225

def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    indices = np.random.permutation(data.shape[0])
    
    adaptation_indices = indices[:360]
    evaluation_indices = indices[360:]
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # print("modified version")
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  # valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  # cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  flow = Sequential(      

# very slow in sampling 
    inv_flow(Sigmoid)(dim=dim),
    BatchNorm(dim=dim),
    # MADE(DSF(dim=dim), cond_dim=cond_dim),
    # Affine(dim=dim),  

      # BatchNorm(dim=dim),
      MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
  ).to(device)


  train_losses, val_losses = train(flow, trainX, trainX ,cond_train=cond_train, cond_val=cond_train , patience=100 ,batch_size=8,n_epochs=1000)
 
  test_labels = np.random.choice(5, size=(400))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(400,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (400,1,28,28)) , test_labels



ways=5
shots=1
meta_lr=0.003
fast_lr=0.5
meta_batch_size=8
adaptation_steps=1
num_iterations=15
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )



device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
# model = l2l.vision.models.OmniglotCNN(ways)
model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels  , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # # Compute meta-validation loss
        # batch = tasksets.validation.sample()
        
        # learner = maml.clone()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
        #                                                     device)
        # meta_valid_error += evaluation_error.item()
        # meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    # print('Meta Valid Error', meta_valid_error / meta_batch_size)
    # print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


105it [00:16,  6.29it/s, epoch_progress=100%, train_loss=-9.641e+03, last_val_loss=+1.381e+05, best_epoch=5, best_loss=-6.126e+03]
105it [00:15,  6.66it/s, epoch_progress=100%, train_loss=-1.221e+04, last_val_loss=+4.146e+05, best_epoch=5, best_loss=-7.486e+03]
105it [00:16,  6.29it/s, epoch_progress=100%, train_loss=-1.143e+04, last_val_loss=+2.669e+05, best_epoch=5, best_loss=-6.419e+03]
106it [00:17,  6.05it/s, epoch_progress=100%, train_loss=-1.197e+04, last_val_loss=+2.994e+05, best_epoch=6, best_loss=-5.869e+03]
104it [00:14,  6.94it/s, epoch_progress=100%, train_loss=-1.110e+04, last_val_loss=+5.896e+05, best_epoch=4, best_loss=-6.736e+03]
107it [00:18,  5.83it/s, epoch_progress=100%, train_loss=-9.927e+03, last_val_loss=+1.031e+05, best_epoch=7, best_loss=-5.997e+03]
104it [00:15,  6.56it/s, epoch_progress=100%, train_loss=-1.013e+04, last_val_loss=+1.791e+05, best_epoch=4, best_loss=-5.866e+03]
104it [00:15,  6.55it/s, epoch_progress=100%, train_loss=-1.132e+04, last_val_loss=



Iteration 0
Meta Train Error 1.616212084889412
Meta Train Accuracy 0.20625000447034836


106it [00:16,  6.36it/s, epoch_progress=100%, train_loss=-1.080e+04, last_val_loss=+1.639e+05, best_epoch=6, best_loss=-6.650e+03]
104it [00:14,  6.96it/s, epoch_progress=100%, train_loss=-1.113e+04, last_val_loss=+4.536e+05, best_epoch=4, best_loss=-6.677e+03]
104it [00:15,  6.61it/s, epoch_progress=100%, train_loss=-1.188e+04, last_val_loss=+4.196e+05, best_epoch=4, best_loss=-6.680e+03]
106it [00:16,  6.38it/s, epoch_progress=100%, train_loss=-1.077e+04, last_val_loss=+2.781e+05, best_epoch=6, best_loss=-5.766e+03]
107it [00:15,  7.07it/s, epoch_progress=100%, train_loss=-1.015e+04, last_val_loss=+9.564e+04, best_epoch=7, best_loss=-5.380e+03]
103it [00:14,  6.88it/s, epoch_progress=100%, train_loss=-1.073e+04, last_val_loss=+2.696e+05, best_epoch=3, best_loss=-6.607e+03]
105it [00:15,  6.64it/s, epoch_progress=100%, train_loss=-1.341e+04, last_val_loss=+4.837e+05, best_epoch=5, best_loss=-7.059e+03]
105it [00:16,  6.30it/s, epoch_progress=100%, train_loss=-1.037e+04, last_val_loss=

...



Iteration 12
Meta Train Error 1.610784962773323
Meta Train Accuracy 0.21250000409781933


105it [00:15,  6.95it/s, epoch_progress=100%, train_loss=-1.130e+04, last_val_loss=+7.373e+04, best_epoch=5, best_loss=-6.591e+03]
103it [00:14,  6.88it/s, epoch_progress=100%, train_loss=-1.309e+04, last_val_loss=+3.452e+05, best_epoch=3, best_loss=-7.502e+03]
105it [00:15,  6.65it/s, epoch_progress=100%, train_loss=-1.134e+04, last_val_loss=+2.487e+05, best_epoch=5, best_loss=-6.862e+03]
104it [00:15,  6.63it/s, epoch_progress=100%, train_loss=-1.079e+04, last_val_loss=+2.153e+05, best_epoch=4, best_loss=-6.597e+03]
102it [00:14,  7.18it/s, epoch_progress=100%, train_loss=-1.114e+04, last_val_loss=+2.111e+05, best_epoch=2, best_loss=-5.955e+03]
106it [00:16,  6.38it/s, epoch_progress=100%, train_loss=-1.107e+04, last_val_loss=+1.910e+05, best_epoch=6, best_loss=-6.812e+03]
105it [00:15,  6.66it/s, epoch_progress=100%, train_loss=-1.126e+04, last_val_loss=+2.060e+05, best_epoch=5, best_loss=-6.572e+03]
104it [00:15,  6.59it/s, epoch_progress=100%, train_loss=-1.269e+04, last_val_loss=



Iteration 13
Meta Train Error 1.592513233423233
Meta Train Accuracy 0.21875000558793545


103it [00:14,  6.90it/s, epoch_progress=100%, train_loss=-1.304e+04, last_val_loss=+3.399e+05, best_epoch=3, best_loss=-7.629e+03]
105it [00:15,  6.66it/s, epoch_progress=100%, train_loss=-1.208e+04, last_val_loss=+6.133e+05, best_epoch=5, best_loss=-6.916e+03]
105it [00:15,  6.63it/s, epoch_progress=100%, train_loss=-1.214e+04, last_val_loss=+2.600e+05, best_epoch=5, best_loss=-7.211e+03]
106it [00:15,  6.69it/s, epoch_progress=100%, train_loss=-9.933e+03, last_val_loss=+1.629e+05, best_epoch=6, best_loss=-6.697e+03]
104it [00:14,  6.99it/s, epoch_progress=100%, train_loss=-1.120e+04, last_val_loss=+3.171e+05, best_epoch=4, best_loss=-6.939e+03]
105it [00:15,  6.65it/s, epoch_progress=100%, train_loss=-1.270e+04, last_val_loss=+2.930e+05, best_epoch=5, best_loss=-7.124e+03]
105it [00:15,  6.67it/s, epoch_progress=100%, train_loss=-1.322e+04, last_val_loss=+3.719e+05, best_epoch=5, best_loss=-7.621e+03]
105it [00:16,  6.33it/s, epoch_progress=100%, train_loss=-1.276e+04, last_val_loss=



Iteration 14
Meta Train Error 1.620359867811203
Meta Train Accuracy 0.22812500596046448


In [ ]:
#@title original code from stage 1 with 400 generated images instead of 1 shot per class
acc_acuracy=[]
iterations= 600
for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))
print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

Mean accuracy/loss, stddev, and confidence intervals
(0.5434166772446285, 0.10080092526546676, 0.008065753861693276)


In [ ]:
print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

0.27500000409781933
0.8250000067055225
